In [1]:
import pandas as pd
import numpy as np
from collections import Counter

In [3]:
anime = pd.read_csv("anime.csv")
rating = pd.read_csv("rating.csv")

# anime dataset description

In [24]:
anime.describe().astype(int)

,anime_id,rating,members
count,12294,12064,12294
mean,14058,6,18071
std,11455,1,54820
min,1,1,5
25%,3484,5,225
50%,10260,6,1550
75%,24794,7,9437
max,34527,10,1013917


In [13]:
anime.columns

Index(['anime_id', 'name', 'genre', 'type', 'episodes', 'rating', 'members'], dtype='object')

In [12]:
anime.groupby('type').count().sort_values("anime_id")

,anime_id,name,genre,episodes,rating,members
type,,,,,,
Music,488,488,488,488,488,488
ONA,659,659,655,659,652,659
Special,1676,1676,1674,1676,1671,1676
Movie,2348,2348,2306,2348,2297,2348
OVA,3311,3311,3310,3311,3285,3311
TV,3787,3787,3777,3787,3671,3787


In [25]:
anime.groupby('genre').count().sort_values("anime_id").iloc[-10:]

,anime_id,name,type,episodes,rating,members
genre,,,,,,
Drama,107,107,105,107,104,107
"Comedy, Kids",112,112,112,112,112,112
Fantasy,114,114,114,114,110,114
"Fantasy, Kids",128,128,128,128,128,128
Dementia,137,137,137,137,137,137
"Comedy, Slice of Life",179,179,179,179,174,179
Kids,199,199,199,199,197,199
Music,301,301,300,301,297,301
Comedy,523,523,523,523,521,523


In [23]:
anime["episodes"].replace('Unknown', np.nan).dropna().astype(int).describe().astype(int)

count    11954
mean        12
std         46
min          1
25%          1
50%          2
75%         12
max       1818
Name: episodes, dtype: int64

In [26]:
anime["episodes"].replace('Unknown', np.nan).isna().sum()

340

# rating dataset description

In [30]:
rating.describe().astype(int)

,user_id,anime_id,rating
count,7813737,7813737,7813737
mean,36727,8909,6
std,20997,8883,3
min,1,1,-1
25%,18974,1240,6
50%,36791,6213,7
75%,54757,14093,9
max,73516,34519,10
